## Agentic RAG System with Smolagents and Chroma DB
## Overview
This notebook presents the development of an Agentic Retrieval-Augmented Generateion (RAG) system powered by smolagents and Chroma Db. The system is designed to provide intelligent access to a custom knowledge base tailored for specialized tasks. While this example focuses on a finance domain use case, the framework is adaptable to any specific domain requiring dynamic data retrieval and classification.

## Components
- **Vector Store:**: We utillize  Chroma DB as the vector storage solution to efficiently house and manage our custom knowledge base in a searchable, vectorized format. 
- **Agentic Search**: We implement an AI agent leveraging semantic search capabilities to enhance retrieval accuracy, enabling the system to refine and optimize its responses based on contextual understanding.


This project showcases a flexible, evolving approach to knowledge management and retrieval, combining cutting-edge vector storage with agent-driven intelligence.

In [1]:
import datasets

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

In [3]:
knowledge_base = datasets.load_dataset("4DR1455/finance_questions", split="train")

In [4]:
knowledge_small = knowledge_base.select(range(500))

In [5]:
knowledge_small

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 500
})

In [6]:
source_docs = [
    Document(page_content=doc["output"], metadata={"source": "finance_questions"})
    for doc in knowledge_small
]


In [7]:
# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
new_docs = text_splitter.split_documents(documents=source_docs)

In [8]:
len(new_docs)

4549

In [9]:
###  BGE Embddings

from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,  encode_kwargs=encode_kwargs
)

/var/folders/79/rc9c84z56hj_4pd56trr0lf00000gn/T/ipykernel_969/1321652950.py:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


In [10]:
import chromadb

In [11]:
db = Chroma.from_documents(new_docs, embeddings)

In [12]:
retriever = db.as_retriever(search_kwargs={"k": 3})
retriever.invoke('credit migration')

[Document(metadata={'source': 'finance_questions', 'start_index': 0}, page_content="Credit migration strategy is a type of credit risk management strategy that focuses on the changes in the credit quality of individual bonds or bond issuers over time. This strategy is based on the principle that the credit quality of bonds and bond issuers can change, or migrate, over time due to various factors such as changes in the financial health of the issuer, changes in macroeconomic conditions, or changes in the issuer's industry."),
 Document(metadata={'source': 'finance_questions', 'start_index': 2440}, page_content='In conclusion, credit migration strategy is a dynamic approach to credit risk management that can help portfolio managers manage interest rate risk. However, it requires a deep understanding of credit markets and the factors that influence credit ratings, as well as the ability to accurately anticipate changes in interest rates.'),
 Document(metadata={'source': 'finance_questions

### Create an Agent that Can Use Tools

In [13]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self,  **kwargs):
        super().__init__(**kwargs)
        self.retriever = db.as_retriever(search_kwargs={"k": 4})

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

retriever_tool = RetrieverTool()

In [14]:
import os

In [15]:
from smolagents import HfApiModel, CodeAgent
# initialize the agetn
agent = CodeAgent(
            tools=[retriever_tool], 
            model=HfApiModel(token=os.environ["HF_TOKEN"]), max_steps=4
        )

In [16]:
agent_output = agent.run("What is credit migration?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is credit migration?                                                                                       │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = retriever(query="explanation of credit migration")                                                      
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved documents:


===== Document 0 =====
Credit migration strategy is a type of credit risk management strategy that focuses on the changes in the credit 
quality of individual bonds or bond issuers over time. This strategy is based on the principle that the credit 
quality of bonds and bond issuers can change, or migrate, over time due to various factors such as changes in the 
financial health of the issuer, changes in macroeconomic conditions, or changes in the issuer's industry.

===== Document 1 =====
In conclusion, credit migration strategy is a dynamic approach to credit risk management that can help portfolio 
managers manage interest rate risk. However, it requires a deep understanding of credit markets and the factors 
that influence credit ratings, as well as the ability to accurately anticipate changes in interest rates.

===== Document 2 =====
Credit migration strategy can be used to manage interest rate risk in the following ways:

1. Diversification: By diversifying the portfolio across different credit ratings, the portfolio manager can reduce
the impact of changes in interest rates on any single bond or group of bonds. This is because different credit 
ratings can react differently to changes in interest rates.

===== Document 3 =====
4. Credit Risk Transfer: CDS contracts facilitate the transfer of credit risk from one party to another. For 
example, banks or financial institutions may sell CDS protection on their loan portfolios to reduce their capital 
requirements and transfer the credit risk to CDS buyers.

Step 4: Mechanics of Credit Default Swaps
To understand the mechanics of a credit default swap, consider the following steps:

Out: None

[Step 1: Duration 4.30 seconds| Input tokens: 2,090 | Output tokens: 91]

...

Execution logs:

Retrieved documents:


===== Document 0 =====
Credit migration strategy is a type of credit risk management strategy that focuses on the changes in the credit 
quality of individual bonds or bond issuers over time. This strategy is based on the principle that the credit 
quality of bonds and bond issuers can change, or migrate, over time due to various factors such as changes in the 
financial health of the issuer, changes in macroeconomic conditions, or changes in the issuer's industry.

===== Document 1 =====
In conclusion, credit migration strategy is a dynamic approach to credit risk management that can help portfolio 
managers manage interest rate risk. However, it requires a deep understanding of credit markets and the factors 
that influence credit ratings, as well as the ability to accurately anticipate changes in interest rates.

===== Document 2 =====
Credit migration strategy can be used to manage interest rate risk in the following ways:

1. Diversification: By diversifying the portfolio across different credit ratings, the portfolio manager can reduce
the impact of changes in interest rates on any single bond or group of bonds. This is because different credit 
ratings can react differently to changes in interest rates.

===== Document 3 =====
Step 1: Understanding the Concept of Recovery Rate
The recovery rate is a measure used in credit risk analysis to assess the potential loss that a lender or investor 
may incur in the event of default by a borrower. It represents the percentage of the outstanding debt that can be 
recovered through the liquidation or restructuring of the borrower's assets.

Out: None

[Step 4: Duration 5.94 seconds| Input tokens: 11,368 | Output tokens: 379]

Reached max steps.

[Step 5: Duration 12.47 seconds| Input tokens: 13,423 | Output tokens: 481]

Final output:
Credit migration refers to the change in the credit quality or credit rating of a borrower, bond issuer, or financial instrument over time. This migration can occur due to various factors, including changes in the financial health of the issuer, shifts in macroeconomic conditions, or changes in the issuer's industry. In the context of credit risk management, credit migration is an important concept because it helps assess potential changes in the creditworthiness of entities and the associated risks, allowing stakeholders to adjust their strategies and portfolios accordingly.


In [17]:
agent.run("Tell me about investing in commodities")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about investing in commodities                                                                          │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  commodities_info = retriever(query="investing in commodities")                                                   
  print(commodities_info)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved documents:


===== Document 0 =====
Overall, investing in commodities carries various risks that investors should carefully assess and manage. 
Understanding the factors that influence commodity prices, monitoring market conditions, and diversifying 
investments can help mitigate these risks and improve the chances of achieving desired investment outcomes.

===== Document 1 =====
Investing in commodities carries certain risks that investors should be aware of. These risks include:

1. Price Volatility: Commodities tend to be highly volatile in terms of price movements. Factors such as supply and
demand imbalances, geopolitical events, weather conditions, and economic indicators can all affect commodity 
prices. This volatility can lead to significant price fluctuations, which may result in substantial gains or losses
for investors.

===== Document 2 =====
2. Market Risk: Commodities are traded on various exchanges, and their prices are influenced by global market 
dynamics. Changes in interest rates, inflation rates, currency exchange rates, and overall market sentiment can all
impact commodity prices. Investors need to be aware of these market risks and understand how they can affect their 
investments.

===== Document 3 =====
. Understanding the relationship between inflation and commodities is crucial for investors in assessing the risks 
and potential returns of commodity investments.

Out: None

[Step 1: Duration 3.02 seconds| Input tokens: 2,091 | Output tokens: 64]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  summary = "Investing in commodities carries various risks, including price volatility and market risk, which     
  investors should carefully assess and manage. Price volatility is due to factors such as supply and demand       
  imbalances, geopolitical events, weather conditions, and economic indicators. Market risk is influenced by       
  global market dynamics, including changes in interest rates, inflation rates, currency exchange rates, and       
  overall market sentiment. Understanding these factors and diversifying investments can help mitigate these       
  risks and potentially improve investment outcomes."                                                              
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Investing in commodities carries various risks, including price volatility and market risk, 
which investors should carefully assess and manage. Price volatility is due to factors such as supply and demand 
imbalances, geopolitical events, weather conditions, and economic indicators. Market risk is influenced by global 
market dynamics, including changes in interest rates, inflation rates, currency exchange rates, and overall market 
sentiment. Understanding these factors and diversifying investments can help mitigate these risks and potentially 
improve investment outcomes.

[Step 2: Duration 7.42 seconds| Input tokens: 4,575 | Output tokens: 213]

'Investing in commodities carries various risks, including price volatility and market risk, which investors should carefully assess and manage. Price volatility is due to factors such as supply and demand imbalances, geopolitical events, weather conditions, and economic indicators. Market risk is influenced by global market dynamics, including changes in interest rates, inflation rates, currency exchange rates, and overall market sentiment. Understanding these factors and diversifying investments can help mitigate these risks and potentially improve investment outcomes.'